# Backtesting a 0DTE Bull Put Spread Strategy Using Historical Market Data

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alpacahq/alpaca-py/blob/master/examples/options/options-zero-dte-backtest.ipynb)

### Strategy Overview
- **Sell a higher-strike put** (collect premium)
- **Buy a lower-strike put** (limit risk)  
- **Net result**: Receive credit upfront, profit if underlying stays above short strike

### Key Assumptions
- No early assignment of OTM short put options
- Perfect liquidity (can exit at any time)
- Uses mid-point pricing of bid-ask spreads
- Single-day trading only (0DTE)

### How the Selection Process Works
The process used in this notebook scans through historical market data chronologically at **one-minute intervals** to find the first valid option pair that meets our trading criteria, holds it until an exit condition is met, then repeats every minute:
1. **Delta-based Selection**:
   - Short Put: Higher delta (closer to ATM) - generates premium income
   - Long Put: Lower delta (further OTM) - provides downside protection

2. **Spread Width Validation**:
   - Ensures risk/reward ratio stays within acceptable bounds
   - Configurable range (default: $2-$4) balances premium vs. risk

3. **Chronological Priority**:
   - Selects the first valid pair found in time sequence
   - Mimics real-world trading where timing matters

### Environment Configuration
- The notebook automatically detects whether it's running in Google Colab or a local IDE
- API keys are loaded differently based on the environment:
   - **Google Colab**: Uses [Colab's Secrets feature](https://x.com/GoogleColab/status/1719798406195867814) (set keys in the left sidebar) 
   - **Local IDE (e.g. VS Code, PyCharm)**: Uses environment variables from a `.env` file
- **Required API Keys**: You'll need to obtain and configure:
   - `ALPACA_API_KEY` and `ALPACA_SECRET_KEY` from your [Alpaca dashboard](https://app.alpaca.markets/dashboard/overview)
   - `DATABENTO_API_KEY` from your [Databento dashboard](https://databento.com/portal/keys)

### Data Sources
Dual API Integration for Comprehensive Market Data:
- **Alpaca Trading API**: Stock daily bars + 1-minute intraday data
- **Databento API**: Options 1-minute tick data with bid/ask spreads

This combination provides both the underlying stock context and detailed options pricing needed for accurate 0DTE backtesting, leveraging each platform's strengths.


In [ ]:
!pip install pandas numpy scipy python-dotenv databento matplotlib alpaca-py

In [ ]:
import os
import sys
from datetime import date, datetime, time, timedelta
from typing import Dict, List, Optional, Tuple
from zoneinfo import ZoneInfo

import databento as db
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from scipy.optimize import brentq
from scipy.stats import norm

from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.trading.client import TradingClient
from alpaca.trading.enums import ContractType
from alpaca.trading.requests import GetCalendarRequest

## Step1: Setting UP the Environment and Trade Parameters

In [ ]:
if 'google.colab' in sys.modules:
    # In Google Colab environment, we will fetch API keys from Secrets.
    # Please set ALPACA_API_KEY, ALPACA_SECRET_KEY, DATABENTO_API_KEY in Google Colab's Secrets from the left sidebar
    from google.colab import userdata
    ALPACA_API_KEY = userdata.get("ALPACA_API_KEY")
    ALPACA_SECRET_KEY = userdata.get("ALPACA_SECRET_KEY")
    DATABENTO_API_KEY = userdata.get("DATABENTO_API_KEY")
else:
    # Please safely store your API keys and never commit them to the repository (use .gitignore)
    # Load environment variables from environment file (e.g., .env)
    load_dotenv()
    # API credentials for Alpaca's Trading API and Databento API
    ALPACA_API_KEY = os.environ.get('ALPACA_API_KEY')
    ALPACA_SECRET_KEY = os.environ.get('ALPACA_SECRET_KEY')
    DATABENTO_API_KEY = os.getenv("DATABENTO_API_KEY")

## We use paper environment for this example
ALPACA_PAPER_TRADE = True # Please do not modify this. This example is for paper trading only.

# Below are the variables for development this documents (Please do not change these variables)
TRADE_API_URL = None
TRADE_API_WSS = None
DATA_API_URL = None
OPTION_STREAM_DATA_WSS = None

In [ ]:
# Initialize Alpaca clients
trade_client = TradingClient(api_key=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY, paper=ALPACA_PAPER_TRADE)
option_historical_data_client = OptionHistoricalDataClient(api_key=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY)
stock_data_client = StockHistoricalDataClient(api_key=ALPACA_API_KEY, secret_key=ALPACA_SECRET_KEY)

In [ ]:
# Initialize Databento client
databento_client = db.Historical(DATABENTO_API_KEY)

In [ ]:
# Underlying symbol
underlying_symbol = 'SPY'

# Risk free rate for the options greeks and IV calculations
RISK_FREE_RATE = 0.01

# Set the timezone
NY_TZ = ZoneInfo('America/New_York')

# Get current date in US/Eastern timezone
today = datetime.now(NY_TZ).date()
start_date = today - timedelta(days=3) # Start of backtesting period: 8 trading days of 0DTE strategy data
end_date = today - timedelta(days=2)   # End of backtesting period: excludes recent data

# Set buffer percentage for the strike price range
BUFFER_PCT = 0.05

# Define delta thresholds
SHORT_PUT_DELTA_RANGE = (-0.60, -0.20) # Short put selection: delta between -0.60 to -0.20 for bull put spread
LONG_PUT_DELTA_RANGE = (-0.40, -0.20) # Long put selection: delta between -0.40 to -0.20 for bull put spread

# Set stop loss threshold threshold (2.5 times of the initial delta)
DELTA_STOP_LOSS_THRES = 2.5

# Set target profit and stop-loss levels (50% of the initial credit)
TARGET_STOP_LOSS_PERCENTAGE = 0.5

## Step2: Getting Historical Stock Daily Bar Data

**Purpose**: Optimize data retrieval by identifying daily price ranges. We use these daily price boundaries (with a small buffer called `buffer_pct`). This allows us to focus on realistic strike prices near the market price (where bull put spreads are typically placed) rather than downloading all available one-minute option contracts and equities bars data.

* The `get_daily_stock_bars_df` function fetches daily price bars for the underlying stock (e.g., SPY) within the specified date range using Alpaca's API.

In [ ]:
def get_daily_stock_bars_df(symbol: str, start_date: date, end_date: date) -> pd.DataFrame:
    """
    Retrieve daily stock bars and replace timestamps with actual market close times.
    
    Used to establish daily high/low price boundaries for option strike selection.
    Gets actual market hours from Alpaca's trading calendar to handle early/late market closes.
    
    Args:
        symbol: Stock ticker symbol (e.g., 'SPY')
        start_date: Start date for data retrieval
        end_date: End date for data retrieval
    
    Returns:
        pd.DataFrame: Daily OHLCV data with expiration_datetime index containing 
                     actual market close times from Alpaca's trading calendar
    """

    req = StockBarsRequest(
        symbol_or_symbols=symbol,
        timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Day),
        start=start_date,
        end=end_date,
    )
    resp = stock_data_client.get_stock_bars(req)
    bars = resp.data[symbol]

    rows = [b.model_dump() for b in bars]
    df = pd.DataFrame(rows)

    # Get market calendar for the same date range
    calendar_request = GetCalendarRequest(start=start_date, end=end_date)
    calendar = trade_client.get_calendar(calendar_request)
    market_close_lookup = {cal.date: cal.close for cal in calendar}

    # Replace timestamp with actual market close datetime as index
    df['expiration_datetime'] = df['timestamp'].dt.date.map(market_close_lookup.get)
    df = df.drop(columns=['timestamp']).set_index("expiration_datetime").sort_index()

    return df

In [ ]:
stock_bars_data = get_daily_stock_bars_df(symbol=underlying_symbol, start_date=start_date, end_date=end_date)

In [ ]:
# Get the first 2 rows of the stock bars data
stock_bars_data.head(2)

## Step3: Retrieving Historical Minute-by-Minute Stock and Options Data by Expiration

**Purpose**: This section generates the universe of relevant put option contracts for backtesting by creating option symbols within a realistic strike price range (±5% buffer around daily high/low prices) and grouping by expiration date based on daily stock bars data. This targeted approach ensures we only fetch option data that traders would realistically consider for bull put spreads, rather than downloading all available contracts.

* `calculate_strike_price_range` calculates the range of option strike prices to consider based on the stock's daily high/low prices with a buffer percentage.
* `generate_put_option_symbols` uses the strike price range to generate standardized option symbol strings for put options within the specified range.
* `collect_option_symbols_by_expiration` creates a list of all relevant option symbols grouped by expiration date for simulating the 0DTE strategy.

In [ ]:
def calculate_strike_price_range(high_price: float, low_price: float, buffer_pct: float = 0.05) -> Tuple[float, float]:
    """
    Calculate strike price boundaries with buffer around daily high/low prices.

    Args:
        high_price: Daily high price
        low_price: Daily low price  
        buffer_pct: Percentage buffer to expand range (default 5%)

    Returns:
        Tuple[float, float]: (min_strike, max_strike) boundaries
    """
    min_strike = low_price * (1 - buffer_pct)
    max_strike = high_price * (1 + buffer_pct)

    return min_strike, max_strike

In [ ]:
for index, row in stock_bars_data.iterrows():
    print(row)
    min_strike, max_strike = calculate_strike_price_range(row['high'], row['low'], buffer_pct=BUFFER_PCT)
    print(f"{index.date()}: min_strike=${min_strike:.2f}, max_strike=${max_strike:.2f}")

In [ ]:
def generate_put_option_symbols(underlying: str, expiration_datetime: datetime, min_strike: float, max_strike: float, strike_increment: float = 1) -> List[str]:
    """
    Generate put option symbols for the given parameters.

    Uses ceiling function to round min_strike UP to nearest whole dollar
    (e.g., $590.96 becomes $591) since options usually trade at integer strikes.

    Args:
        underlying: Underlying symbol (e.g., 'SPY')
        expiration_datetime: Option expiration datetime
        min_strike: Minimum strike price
        max_strike: Maximum strike price
        strike_increment: Strike price increment (default 1)

    Returns:
        List[str]: Formatted option symbols (e.g., 'SPY250616P00571000')
    """
    option_symbols = []

    # Format expiration datetime as date: YYMMDD
    expiration_date = expiration_datetime.date()
    exp_str = expiration_date.strftime("%y%m%d")

    # Generate strikes in increments (rounds UP to the nearest integer)
    current_strike = np.ceil(min_strike / strike_increment) * strike_increment

    while current_strike <= max_strike:
        # Format strike price as 8-digit integer (multiply by 1000)
        strike_formatted = f"{int(current_strike * 1000):08d}"

        # Create option symbol: SPY + YYMMDD + P + 8-digit strike
        option_symbol = f"{underlying}{exp_str}P{strike_formatted}"
        option_symbols.append(option_symbol)

        current_strike += strike_increment

    return option_symbols

In [ ]:
def collect_option_symbols_by_expiration(stock_bars_data: pd.DataFrame, underlying_symbol: str, buffer_pct: float) -> Dict[pd.Timestamp, List[str]]:

    """
    Collect option symbols grouped by expiration datetime for 0DTE backtesting.
    
    For each trading day in the stock bars data, generates put option symbols
    within a realistic strike range and groups them by expiration date.

    Args:
        stock_bars_data: DataFrame containing daily OHLCV data with timestamp index
        underlying_symbol: Stock symbol (e.g., 'SPY') for option generation
        buffer_pct: Buffer percentage to expand strike range around daily high/low

    Returns:
        Dict[pd.Timestamp, List[str]]: Dictionary mapping expiration datetime (e.g., Timestamp('2025-07-21 16:00:00')) 
                                       to lists of option symbols for that date
    """
    # Collect all symbols
    option_symbols_by_expiration = {}

    for index, row in stock_bars_data.iterrows():
        min_strike, max_strike = calculate_strike_price_range(row['high'], row['low'], buffer_pct=buffer_pct)

        option_symbols = generate_put_option_symbols(underlying_symbol, expiration_datetime=index, min_strike=min_strike, max_strike=max_strike, strike_increment=1)

        # Group symbols by expiration date
        if index not in option_symbols_by_expiration:
            option_symbols_by_expiration[index] = []

        option_symbols_by_expiration[index].extend(option_symbols)

    return option_symbols_by_expiration

In [ ]:
option_symbols_by_expiration = collect_option_symbols_by_expiration(stock_bars_data, underlying_symbol, buffer_pct=BUFFER_PCT)

In [ ]:
# Show just the keys (dates) to understand the structure
keys = list(option_symbols_by_expiration.keys())
keys

In [ ]:
# Compare calculated vs generated strike ranges. You can see that the generated strike ranges are within the calculated range.
first_day = stock_bars_data.head(1).iloc[0]
calc_min, calc_max = first_day['low'] * (1-BUFFER_PCT), first_day['high'] * (1+BUFFER_PCT)
print(f"Calculated range: ${calc_min:.2f} - ${calc_max:.2f}")

symbols = option_symbols_by_expiration[keys[0]] # Show the first expiration date's option symbols
symbols[:2] + ['...'] + symbols[-2:] # Show first 2 and last 2 option symbols to see the strike range

* The `get_stock_and_option_historical_data` function uses the collection of all relevant option symbols grouped by expiration timestamp from the `collect_option_symbols_by_expiration` function and retrieves minute-by-minute market data for both stock and options using Alpaca and Databento APIs, providing the data needed for realistic backtesting.

In [ ]:
def extract_strike_price_from_symbol(symbol: str) -> float:
    """
    Extract strike price from option symbol.
    Converts last 8 digits of option symbol to strike price by dividing by 1000.
    
    Args:
        symbol: Option symbol (e.g., 'SPY250616P00571000')
    
    Returns:
        float: Strike price (e.g., 571.0) or 0.0 if invalid format
    """
    # Option symbols typically have format: TICKER + YYMMDD + (C/P) + 8-digit strike price
    # The last 8 digits represent strike price * 1000
    try:
        # Extract the last 8 digits and convert to strike price
        strike_str = symbol[-8:]
        strike_price = float(strike_str) / 1000.0
        return strike_price
    except (ValueError, IndexError):
        print(f"Warning: Could not extract strike price from symbol {symbol}")
        return 0.0

In [ ]:
# def get_stock_and_option_historical_data(option_symbols_by_expiration: Dict[datetime.date, List[str]], underlying_symbol: str) -> Dict[datetime, List[Dict]]:
#     """
#     Get option tick data (default 1 minutes time interval) for the specified symbols and date range.
#     The start and end datetime are set to 13:30 and 20:00 for the option tick data.

#     Args:
#         option_symbols_by_expiration:
#         underlying_symbol:

#     Returns:
#         Dictionary with timestamp as key and list of option data dictionaries as value
#     """
#     unsorted_stock_option_historical_data_by_timestamp = {}

#     for expiration_date, symbols_list in option_symbols_by_expiration.items():

#         # Create start and end datetime with specific times and timezone
#         start_datetime = datetime(expiration_date.year, expiration_date.month, expiration_date.day, 9, 30, tzinfo=NY_TZ)
#         end_datetime = datetime(expiration_date.year, expiration_date.month, expiration_date.day, 16, 15, tzinfo=NY_TZ)
#         print(f"Time range: {start_datetime} to {end_datetime}")

#         # Get stock bar data (1 minute interval) for the specified underlying symbol
#         req = StockBarsRequest(
#             symbol_or_symbols=underlying_symbol,
#             timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Minute),
#             start=start_datetime,
#             end=end_datetime
#         )
#         stock_res = stock_data_client.get_stock_bars(req)

#         # Create a dictionary of stock close prices by timestamp for quick lookup
#         stock_close_by_timestamp = {}
#         if underlying_symbol in stock_res.data:
#             for bar in stock_res.data[underlying_symbol]:
#                 stock_close_by_timestamp[bar.timestamp] = bar.close
#             print(f"Retrieved 1-minute stock bars for {underlying_symbol}: {len(stock_resp.data[underlying_symbol])} total bars")
#         # # Create a dictionary of stock close prices by timestamp for quick lookup
#         # stock_close_by_timestamp = {}

#         # if underlying_symbol in stock_resp.data:
#         #     for stock_bar in stock_resp.data[underlying_symbol]:
#         #         stock_dict = stock_bar.model_dump()
#         #         stock_close_by_timestamp[stock_dict['timestamp']] = stock_dict['close']
#         #     print(f"Retrieved data for {underlying_symbol} stock: {len(stock_resp.data[underlying_symbol])} total bars")

#         # Transform options symbols to format required by databento (add spaces)
#         formatted_option_symbols = [f"{symbol[:3]}   {symbol[3:]}" for symbol in symbols_list]

#         # Get option tick data using databento client
#         option_df = databento_client.timeseries.get_range(
#             dataset="OPRA.PILLAR",
#             schema="cbbo-1m",
#             symbols=formatted_option_symbols,
#             start=start_datetime,
#             end=end_datetime,
#         ).to_df()

#         # Convert market close time to UTC
#         market_close_utc = end_datetime.astimezone(ZoneInfo("UTC"))

#         # Process the DataFrame and organize by timestamp
#         if not option_df.empty:
#             for timestamp, row in option_df.iterrows():
#                 # Extract option symbol by removing spaces (e.g. 'SPY   250627C00615000' -> 'SPY250627C00615000')
#                 option_symbol = row['symbol'].replace(' ', '')

#                 # Create tick data dictionary similar to original format
#                 tick_data_dict = {
#                     'option_symbol': option_symbol,
#                     'timestamp': timestamp, # timestamp is the index of the DataFrame
#                     'close': row['price'],
#                     'bid': row['bid_px_00'],
#                     'ask': row['ask_px_00'],
#                     'midpoint': (row['bid_px_00'] + row['ask_px_00']) / 2,  # Mid price
#                     'bid_size': row['bid_sz_00'],
#                     'ask_size': row['ask_sz_00']
#                 }

#                 # Add strike price to the bar dictionary
#                 tick_data_dict['strike_price'] = extract_strike_price_from_symbol(option_symbol)

#                 # Add expiration date to the bar dictionary
#                 tick_data_dict['expiry'] = market_close_utc

#                 # Add stock close price if available for this timestamp
#                 if timestamp in stock_close_by_timestamp:
#                     tick_data_dict['underlying_close'] = stock_close_by_timestamp[timestamp]

#                 # Initialize timestamp key if it doesn't exist
#                 if timestamp not in unsorted_stock_option_historical_data_by_timestamp:
#                     unsorted_stock_option_historical_data_by_timestamp[timestamp] = []

#                 # Add this tick data to the timestamp group
#                 unsorted_stock_option_historical_data_by_timestamp[timestamp].append(tick_data_dict)

#             print(f"Retrieved data for {len(formatted_option_symbols)} options symbols: {len(option_df)} total tick data")
#         else:
#             print(f"No data found for options symbols: {formatted_option_symbols}")

#     # Sort timestamps for consistent ordering
#     # sorted_timestamps = sorted(unsorted_stock_option_historical_data_by_timestamp.keys())
#     # sorted_stock_option_historical_data_by_timestamp = {ts: unsorted_stock_option_historical_data_by_timestamp[ts] for ts in sorted_timestamps}
#     sorted_stock_option_historical_data_by_timestamp = dict(sorted(unsorted_stock_option_historical_data_by_timestamp.items()))

#     return sorted_stock_option_historical_data_by_timestamp


In [ ]:
def get_stock_and_option_historical_data(option_symbols_by_expiration: Dict[pd.Timestamp, List[str]], underlying_symbol: str) -> pd.DataFrame:
    """
    Get combined option and stock data as a DataFrame.
    Returns options data with stock close prices merged by timestamp for easier inspection.

    Args:
        option_symbols_by_expiration: Dictionary mapping dates to option symbol lists
        underlying_symbol: Stock symbol for underlying asset

    Returns:
        pd.DataFrame: Combined options and stock data with renamed columns
    """
    all_data_frames = []

    for expiration_datetime, symbols_list in option_symbols_by_expiration.items():

        # Create start datetime at 9:30 AM for the same date
        start_datetime = expiration_datetime.replace(hour=9, minute=30, second=0, microsecond=0)
        # Use expiration_datetime directly as end_datetime (already contains actual market close)
        end_datetime = expiration_datetime

        # Get stock bar data (1 minute interval) for the specified underlying symbol
        req = StockBarsRequest(
            symbol_or_symbols=underlying_symbol,
            timeframe=TimeFrame(amount=1, unit=TimeFrameUnit.Minute),
            start=start_datetime,
            end=end_datetime
        )
        stock_res = stock_data_client.get_stock_bars(req)

        # Create a dictionary of stock close prices by timestamp for quick lookup
        stock_close_by_timestamp = {}
        if underlying_symbol in stock_res.data:
            for bar in stock_res.data[underlying_symbol]:
                stock_close_by_timestamp[bar.timestamp] = bar.close
            print(f"Retrieved 1-minute stock bars for {underlying_symbol}: {len(stock_res.data[underlying_symbol])} total bars")

            # Transform options symbols to format required by databento (add spaces)
            formatted_option_symbols = [f"{symbol[:3]}   {symbol[3:]}" for symbol in symbols_list]

            # Get option tick data using databento client
            option_df = databento_client.timeseries.get_range(
                dataset="OPRA.PILLAR",
                schema="cbbo-1m",
                symbols=formatted_option_symbols,
                start=start_datetime,
                end=end_datetime,
            ).to_df()

            if not option_df.empty:
                # Add derived columns to option_df
                option_df['option_symbol'] = option_df['symbol'].str.replace(' ', '')
                option_df['strike_price'] = option_df['option_symbol'].apply(extract_strike_price_from_symbol)
                option_df['midpoint'] = (option_df['bid_px_00'] + option_df['ask_px_00']) / 2
                option_df['expiry'] = end_datetime.tz_localize(NY_TZ).astimezone(ZoneInfo("UTC"))
                
                # Map stock close prices using timestamp lookup (handles one-to-many correctly)
                option_df['underlying_close'] = option_df.index.map(stock_close_by_timestamp)
                
                # Select and rename columns as needed
                final_df = option_df[[
                    'option_symbol', 'strike_price', 'price', 'bid_px_00', 'ask_px_00', 
                    'midpoint', 'bid_sz_00', 'ask_sz_00', 'expiry', 'underlying_close'
                ]].rename(columns={
                    'price': 'close',
                    'bid_px_00': 'bid',
                    'ask_px_00': 'ask', 
                    'bid_sz_00': 'bid_size',
                    'ask_sz_00': 'ask_size'
                })
                
                # Reset index to make timestamp a column and rename it
                final_df.reset_index(inplace=True)
                final_df.rename(columns={'ts_recv': 'timestamp'}, inplace=True)
                
                all_data_frames.append(final_df)
                print(f"Retrieved data for {len(formatted_option_symbols)} options symbols: {len(option_df)} total rows")
            else:
                print(f"No data found for options symbols: {formatted_option_symbols}")

        # Combine all DataFrames
        if all_data_frames:
            result_df = pd.concat(all_data_frames, ignore_index=True)
            return result_df.sort_values('timestamp').reset_index(drop=True)
        else:
            return pd.DataFrame()

In [ ]:
stock_option_historical_data_by_timestamp = get_stock_and_option_historical_data(option_symbols_by_expiration, underlying_symbol)

In [ ]:
# Show first 2 and last 2 rows of the stock_option_historical_data_by_timestamp
pd.concat([stock_option_historical_data_by_timestamp.head(2), stock_option_historical_data_by_timestamp.tail(2)])

## Step 4: Implementing Options Greeks and IV Calculations from Historical Option Bars

TODO: write purpose of this step

* The `calculate_delta_historical` function uses `calculate_implied_volatility` and computes the delta of an option using historical data, considering the time to expiry, implied volatility, and option type. It adjusts for options nearing expiration by setting delta based on intrinsic value when necessary.

* The `calculate_implied_volatility` function estimates the implied volatility of an option by solving for the volatility that matches the observed option price, using the Black-Scholes model. It handles edge cases where the option price is close to intrinsic value by returning a near-zero volatility.

In [ ]:
# Calculate implied volatility
def calculate_implied_volatility(option_price, S, K, T, r, option_type) -> float:

    # Define a reasonable range for sigma
    sigma_lower = 1e-6
    sigma_upper = 5.0  # Adjust upper limit if necessary

    # Check if the option is out-of-the-money and price is close to zero
    intrinsic_value = max(0, (S - K) if option_type == 'call' else (K - S))
    if option_price <= intrinsic_value + 1e-6:
        # print("Option price is close to intrinsic value; implied volatility is near zero.") # Uncomment for checking the status
        return 0.0

    # Define the function to find the root
    def option_price_diff(sigma):
        d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
        if option_type == 'call':
            price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
        elif option_type == 'put':
            price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
        return price - option_price

    try:
        return brentq(option_price_diff, sigma_lower, sigma_upper)
    except ValueError as e:
        print(f"Failed to find implied volatility: {e}")
        return None

In [ ]:
# Calculate historical option Delta
def calculate_delta_historical(option_price, strike_price, expiry, underlying_price, risk_free_rate, option_type, timestamp) -> Optional[float]:

    # Calculate the time to expiry in years
    T = (expiry - timestamp).total_seconds() / (365 * 24 * 60 * 60)
    # Set minimum T to avoid zero
    T = max(T, 1e-6)

    if T == 1e-6:
        print("Option has expired or is expiring now; setting delta based on intrinsic value.")
        if option_type == 'put':
            return -1.0 if underlying_price < strike_price else 0.0
        else:
            return 1.0 if underlying_price > strike_price else 0.0

    implied_volatility = calculate_implied_volatility(option_price, underlying_price, strike_price, T, risk_free_rate, option_type)
    if implied_volatility is None:
        print("Implied volatility could not be determined, skipping delta calculation.")
        return None

    d1 = (np.log(underlying_price / strike_price) + (risk_free_rate + 0.5 * implied_volatility ** 2) * T) / (implied_volatility * np.sqrt(T))
    delta = norm.cdf(d1) if option_type == 'call' else -norm.cdf(-d1)
    return delta

## Step 5: Finding Short and Long Puts for a 0DTE Bull Put Spread

This is the main algorithmic trading logic that forms the core of our 0DTE bull put spread options strategy. The `find_short_and_long_puts` function implements the systematic approach to identify and select optimal option pairs for bull put spreads.

**Note**: You can uncomment print functions within `find_short_and_long_puts` to assess the algorithm as well


<b>Algorithm Flow</b>

```json
For each timestamp in historical data (starting from the earliest timestamp):
└── For each option in that timestamp (e.g. '2025-05-10 13:45:00' UTC)
    ├── Calculate option delta
    ├── Check if delta fits short put criteria → Store if match
    ├── Check if delta fits long put criteria → Store if match
    └── If both options found:
        ├── Validate spread width
        ├── If valid → Return pair (STOP)
        └── If invalid → Reset and continue search
```

In [ ]:
def create_option_dict_historical(option_symbol, strike_price, underlying_price, delta, option_price, timestamp, expiry) -> dict:
    """Create option dictionary from historical tick data"""
    return {
        'option_symbol': option_symbol,
        'strike_price': strike_price,
        'underlying_price': underlying_price,
        'delta': delta,
        'option_price': option_price,
        'timestamp': timestamp,
        'expiration_date': expiry,
    }

In [ ]:
def find_short_and_long_puts(
    stock_option_historical_data_by_timestamp,
    risk_free_rate,
    short_put_delta_range: List,
    long_put_delta_range: List,
    spread_width=(2, 4),
    option_type=ContractType.PUT
):
    """
    Identify the short put and long put from the options chain.
    Returns dictionaries containing details of the selected options.

    Args:
        start_date: Start date for the analysis period
        end_date: End date for the analysis period
        stock_option_historical_data_by_timestamp: Historical data dictionary
        risk_free_rate: Risk-free rate for options calculations
        short_put_delta_range: Delta range for short put selection
        long_put_delta_range: Delta range for long put selection
        spread_width: Tuple of (min_width, max_width) for spread
        option_type: Type of option (PUT or CALL)
        underlying_symbol: Symbol of underlying asset
    """
    short_put = None
    long_put = None


    for timestamp, tick_data_list in stock_option_historical_data_by_timestamp.items():
        print(f"Analyzing timestamp: {timestamp}")

        for tick_data in tick_data_list:
            option_symbol = tick_data['option_symbol']
            print(f"Option symbol is: {option_symbol}")

            # Get tick data
            underlying_price = tick_data['underlying_close']
            option_price = tick_data['midpoint']
            strike_price = tick_data['strike_price']
            expiry = tick_data['expiry']
            timestamp = tick_data['timestamp']

            # Calculate delta
            delta = calculate_delta_historical(
                option_price=option_price,
                strike_price=strike_price,
                expiry=expiry,
                underlying_price=underlying_price,
                risk_free_rate=risk_free_rate,
                option_type=option_type,
                timestamp=timestamp
            )
            print(f"Delta for {option_symbol} is: {delta}")
            # Skip this option if delta calculation failed
            if delta is None:
                print(f"Delta calculation failed for {option_symbol} at timestamp: {timestamp}")
                continue

            # Check if this option meets short put criteria
            if not short_put and short_put_delta_range[0] <= delta <= short_put_delta_range[1]:
                short_put = create_option_dict_historical(option_symbol, strike_price, underlying_price, delta, option_price, timestamp, expiry)
                # print(f"Found short put at timestamp: {timestamp}")
            elif long_put_delta_range[0] <= delta <= long_put_delta_range[1]:
                long_put = create_option_dict_historical(option_symbol, strike_price, underlying_price, delta, option_price, timestamp, expiry)
                # print(f"Long put found at timestamp: {timestamp}")

            # Check spread width only when both options are found
            if short_put and long_put:
                current_spread_width = abs(short_put['strike_price'] - long_put['strike_price'])
                if not (spread_width[0] <= current_spread_width <= spread_width[1]):
                    print(f"Spread width of {spread_width} is outside the target range of ${spread_width[0]}-${spread_width[1]}; resetting search.")
                    # Reset both options to continue searching
                    short_put = None
                    long_put = None
                    continue
                else:
                    # Exit immediately when valid pair found
                    print(f"Valid spread found with width ${spread_width} at timestamp: {timestamp} for {short_put['option_symbol']} and {long_put['option_symbol']} at underlying price: {underlying_price}")
                    return short_put, long_put

    return short_put, long_put

In [ ]:
SPREAD_WIDTH = (2, 4)
short_put, long_put = find_short_and_long_puts(stock_option_historical_data_by_timestamp, RISK_FREE_RATE, SHORT_PUT_DELTA_RANGE, LONG_PUT_DELTA_RANGE, SPREAD_WIDTH, option_type=ContractType.PUT)
short_put, long_put

## Step 6: Executing a 0DTE Bull Put Spread Using Historical Stock and Option Bars

The `trade_0DTE_options_historical` function simulates trading a bull put vertical spread on options expiring the same day (0DTE) using historical market data.

### How It Works
1. **Find Options**: Automatically selects short/long puts based on delta ranges and spread width ($2-$5)
2. **Monitor Position**: Tracks option prices and Greeks throughout the trading day
3. **Exit When**: One of four conditions is met (in priority order):

### Exit Conditions
| Condition | Trigger | Result |
|-----------|---------|---------|
| **Profit Target** | Spread price drops to target level | Take profit |
| **Delta Stop Loss** | Position delta exceeds risk threshold | Cut losses |
| **Assignment Risk** | Underlying drops below short strike | Avoid assignment |
| **Expiration** | End of trading day reached | Keep full credit |

In [ ]:
def trade_0DTE_options_historical(
    stock_option_historical_data_by_timestamp,
    risk_free_rate,
    delta_stop_loss_thres,
    target_stop_loss_percentage,
    short_put_delta_range,
    long_put_delta_range,
    spread_width
):
    """
    Execute a 0DTE bull put vertical spread using historical data for backtesting.
    """

    short_put, long_put = find_short_and_long_puts(
        stock_option_historical_data_by_timestamp,
        risk_free_rate,
        short_put_delta_range,
        long_put_delta_range,
        spread_width,
        option_type=ContractType.PUT
    )

    # Extract parameters
    short_symbol = short_put['option_symbol']
    long_symbol = long_put['option_symbol']
    short_strike = short_put['strike_price']
    long_strike = long_put['strike_price']
    expiration_date = short_put['expiration_date']
    entry_timestamp = short_put['timestamp']  # Get entry timestamp from option data
    short_price = short_put['option_price']
    long_price = long_put['option_price']

    # Calculate initial metrics
    credit_received = short_put['option_price'] - long_put['option_price']
    initial_total_delta = abs(short_put['delta']) - abs(long_put['delta'])

    # Calculate delta stop loss (default: 2.5 times of the initial delta)
    delta_stop_loss = initial_total_delta * delta_stop_loss_thres
    # Set target profit price (default: 50% of the initial credit)
    target_profit_price = credit_received * target_stop_loss_percentage

    # Monitor through historical data starting after entry timestamp
    for timestamp in stock_option_historical_data_by_timestamp.keys():
        if timestamp <= entry_timestamp:
            continue  # Skip timestamps up to and including entry time

        option_tick_data_list = stock_option_historical_data_by_timestamp[timestamp]

        # Find current prices
        current_short_price = None
        current_long_price = None
        current_underlying_price = None


        for tick_data in option_tick_data_list:
            # If the the option symbol is same as short_symbol, then replace the current_short_price and current_underlying_price
            if tick_data['option_symbol'] == short_symbol:
                current_short_price = tick_data['midpoint']
                current_underlying_price = tick_data.get('underlying_close')

            # If the the option symbol is same as long_symbol, then replace the current_long_price and current_underlying_price
            elif tick_data['option_symbol'] == long_symbol:
                current_long_price = tick_data['midpoint']
                if current_underlying_price is None:
                    current_underlying_price = tick_data.get('underlying_close')

        # If any of the prices are not found, then skip the timestamp
        if not all([current_short_price, current_long_price, current_underlying_price]):
            # print(f"Prices were not found at timestamp: {timestamp}")
            continue

        current_spread_price = current_short_price - current_long_price

        # Calculate current deltas
        current_short_delta = calculate_delta_historical(current_short_price, short_strike, expiration_date, current_underlying_price, risk_free_rate, 'put', timestamp)
        current_long_delta = calculate_delta_historical(current_long_price, long_strike, expiration_date, current_underlying_price, risk_free_rate, 'put', timestamp)

        if current_short_delta is None or current_long_delta is None:
            # print(f"Delta was not calculated at timestamp: {timestamp}")
            continue

        current_total_delta = abs(current_short_delta) - abs(current_long_delta)

        # Check exit conditions (In live trading, we place the order to exit here)
        # If the spread price is less than the target profit price, then exit and return the profit
        if current_spread_price <= target_profit_price:
            return {
                'status': 'theoretical_profit',
                'theoretical_pnl': (credit_received - current_spread_price) * 100,
                'exit_time': timestamp,
                'short_put_symbol': short_symbol,
                'long_put_symbol': long_symbol,
                'entry_time': entry_timestamp
            }

        # If the total delta is greater than the delta stop loss, then exit and return the stop loss
        if current_total_delta >= delta_stop_loss:
            return {
                'status': 'stop_loss',
                'theoretical_pnl': (credit_received - current_spread_price) * 100,
                'exit_time': timestamp,
                'short_put_symbol': short_symbol,
                'long_put_symbol': long_symbol,
                'entry_time': entry_timestamp
            }

        # If the underlying price is less than the short strike price, then short put is being exercised and calculate the theoretical loss
        if current_underlying_price <= short_strike:
            theoretical_loss = short_price - long_price - short_strike + current_underlying_price
            return {
                'status': 'theoretical_loss',
                'theoretical_pnl': theoretical_loss * 100,
                'exit_time': timestamp,
                'short_put_symbol': short_symbol,
                'long_put_symbol': long_symbol,
                'entry_time': entry_timestamp
            }

    # Handle expiration
    final_timestamp = max(stock_option_historical_data_by_timestamp.keys())

    return {
        'status': 'expired',
        'theoretical_pnl': credit_received,
        'exit_time': final_timestamp,
        'short_put_symbol': short_symbol,
        'long_put_symbol': long_symbol,
        'entry_time': entry_timestamp
    }

## Step 7: Running an Iterative Backtest

* `run_iterative_backtest` coordinates the entire backtesting process, running multiple trades sequentially throughout the historical period, and aggregates all results.

In [ ]:
# Run iterative backtest
def run_iterative_backtest(
    max_iterations,
    underlying_symbol,
    start_date,
    end_date,
    buffer_pct,
    risk_free_rate,
    delta_stop_loss_thres,
    target_stop_loss_percentage,
    short_put_delta_range,
    long_put_delta_range,
    spread_width,
):
    """
    Run iterative backtest that continuously finds and trades new option pairs throughout the day.
    """
    all_results = []
    iteration = 1

    try:
        # Get initial data
        stock_bars_data = get_daily_stock_bars_df(underlying_symbol, start_date, end_date)
        option_symbols_by_expiration = collect_option_symbols_by_expiration(stock_bars_data=stock_bars_data, underlying_symbol=underlying_symbol, buffer_pct=buffer_pct)
        stock_option_historical_data_by_timestamp = get_stock_and_option_historical_data(option_symbols_by_expiration=option_symbols_by_expiration, underlying_symbol=underlying_symbol)

        # Initialize start time (first timestamp in data)
        current_start_time = min(stock_option_historical_data_by_timestamp.keys())

        while True:
            print(f"\n--- Iteration {iteration} ---")
            print(f"Starting from: {current_start_time}")

            # Filter data to only include timestamps after current_start_time
            filtered_historical_stock_and_option_data_by_timestamp = {
                timestamp: bars for timestamp, bars in stock_option_historical_data_by_timestamp.items()
                if timestamp >= current_start_time
            }

            if not filtered_historical_stock_and_option_data_by_timestamp:
                print("No more data available. Ending iterations.")
                break

            # Execute trade (function will find options for bull put spread strategy internally)
            result = trade_0DTE_options_historical(
                filtered_historical_stock_and_option_data_by_timestamp,
                risk_free_rate,
                delta_stop_loss_thres,
                target_stop_loss_percentage,
                short_put_delta_range,
                long_put_delta_range,
                spread_width
            )

            if not result:
                print("Could not execute trade. Ending iterations.")
                break

            # Store result
            result['iteration'] = iteration
            all_results.append(result)

            print(f"Status: {result['status']} | theoretical PnL: ${result['theoretical_pnl']:.2f} | {result['short_put_symbol']} & {result['long_put_symbol']} | Entry Time: {result['entry_time']} | Exit Time: {result['exit_time']}")

            # Update start time for next iteration to be after the exit time
            current_start_time = result['exit_time']

            # Add small buffer to ensure we don't include the exit timestamp
            current_start_time = current_start_time + pd.Timedelta(minutes=1)

            iteration += 1

            # Safety check to prevent infinite loops
            if iteration > max_iterations:  # Adjust as needed
                print("Maximum iterations reached. Stopping.")
                break

    except Exception as e:
        print(f"Error in iteration {iteration}: {e}")

    return all_results

## Step 8: Displaying the Results

In [ ]:
MAX_ITERATIONS = 5000
BUFFER_PCT = 0.05
SPREAD_WIDTH = (2, 4)

# Execute iterative backtest
results = run_iterative_backtest(
    max_iterations=MAX_ITERATIONS,
    underlying_symbol=underlying_symbol,
    start_date=start_date,
    end_date=end_date,
    buffer_pct=BUFFER_PCT,
    risk_free_rate=RISK_FREE_RATE,
    delta_stop_loss_thres=DELTA_STOP_LOSS_THRES,
    target_stop_loss_percentage=TARGET_STOP_LOSS_PERCENTAGE,
    short_put_delta_range=SHORT_PUT_DELTA_RANGE,
    long_put_delta_range=LONG_PUT_DELTA_RANGE,
    spread_width=SPREAD_WIDTH
)

# Display summary
if results:
    total_pnl = sum(result['theoretical_pnl'] for result in results)
    print(f"\n--- Summary ---")
    print(f"Total trades: {len(results)}")
    print(f"Total theoretical P&L: ${total_pnl:.2f}")
    for i, result in enumerate(results, 1):
        print(f"Trade {i}: {result['status']} | ${result['pnl']:.2f}")

In [ ]:
# Convert to DataFrame and calculate cumulative P&L
df = pd.DataFrame(results)
df['cumulative_pnl'] = df['theoretical_pnl'].cumsum()

# Convert entry_time to datetime and extract date
df['entry_time'] = pd.to_datetime(df['entry_time'])

# Create plot
plt.figure(figsize=(12, 6))
plt.plot(df['entry_time'], df['cumulative_pnl'], linewidth=2, marker='o', markersize=4)
plt.axhline(y=0, color='red', linestyle='--', alpha=0.7)
plt.title('Cumulative P&L Over Time', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Cumulative P&L ($)')
plt.grid(True, alpha=0.3)

# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Add final P&L text
final_pnl = df['cumulative_pnl'].iloc[-1]
plt.text(df['entry_time'].iloc[int(len(df)*0.7)], final_pnl, f'Final P&L: ${final_pnl:.2f}',
         fontsize=12, fontweight='bold',
         bbox=dict(boxstyle="round,pad=0.3", facecolor='yellow', alpha=0.7))

plt.tight_layout()
plt.show()

# Simple summary
print(f"\nTotal Trades: {len(df)}")
print(f"Total theoretical P&L: ${df['theoretical_pnl'].sum():.2f}")
# print(f"Win Rate: {(df['pnl'] > 0).mean()*100:.1f}%")